# Demo Notebook for Creating an Inverse Kinematics Report

Run the cells in order, making sure to enter AWS credentials in the cell when prompted


In [ ]:
#@title Install Python Packages
!pip install -q awswrangler > /dev/null
!pip install -q plotly > /dev/null
!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@v2.1.0#egg=reboot_toolkit > /dev/null
!echo "Done Installing"

In [ ]:
#@title Import Statements

import awswrangler as wr
import numpy as np
import os

from getpass import getpass
from plotly.subplots import make_subplots
from typing import Optional, Dict
from dataclasses import dataclass

import reboot_toolkit as rtk
from reboot_toolkit import MovementType, Handedness, MocapType, FileType, setup_aws, S3Metadata

In [ ]:
#@title AWS Credentials

boto3_session = setup_aws(org_id="org-mlbbiomech", aws_default_region="us-west-1")

In [ ]:
#@title S3 DataFrame Functions
    
@dataclass
class PlayerMetadata:
    session_date: str
    game_pk: str
    player_id: str
    play_guid: Optional[str] = None
    s3_metadata: Optional[S3Metadata] = None
        
    @property
    def s3_prefix(self) -> str:
        assert self.s3_metadata, "Must set s3 metadata before generating s3 prefix"
        return f's3://{self.s3_metadata.bucket}/data_delivery/{self.s3_metadata.mocap_type}/{self.session_date}/{self.game_pk}/{self.s3_metadata.movement_type}/{self.player_id}/{self.s3_metadata.file_type}/'


def get_analysis_dict(player_metadata: PlayerMetadata) -> Dict:
    """Construct dict of metadata for generating an analysis."""
    player_dict = {
        'session_date': player_metadata.session_date,
        'game_pk': player_metadata.game_pk,
        'player_id': player_metadata.player_id,
        'play_guid': player_metadata.play_guid,
        'eye_hand_multiplier': player_metadata.s3_metadata.handedness.eye_hand_multiplier,
    }

    player_dict['s3_prefix'] = player_metadata.s3_prefix
  
    return player_dict


def load_data_into_analysis_dict(analysis_dict: Dict) -> None:
    print('Loading player', analysis_dict['player_id'], 'from:', analysis_dict['s3_prefix'])
    print('Downloading data...')
    df = wr.s3.read_csv(analysis_dict['s3_prefix'], index_col=[0])

    print('Aggregating mean data...')    
    df['rel_frame'] = df['time_from_max_hand'].copy()
    df['rel_frame'] = df.groupby('org_movement_id')['rel_frame'].transform(rtk.get_relative_frame)
    analysis_dict['df_mean'] = df.groupby('rel_frame').agg('mean', numeric_only=True).reset_index()
    
    if analysis_dict['play_guid'] is None:
        play_guid = list(df['org_movement_id'].unique())[0]
        analysis_dict['play_guid'] = play_guid
    else:
        play_guid = analysis_dict['play_guid']
    analysis_dict['df'] = df.loc[df['org_movement_id'] == play_guid]

In [ ]:
#@title Set Constant S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=[MocapType.HAWKEYE_HFR],
    file_type=FileType.INVERSE_KINEMATICS,
    movement_type=MovementType.BASEBALL_PITCHING,
    handedness=Handedness.RIGHT,
)

In [ ]:
#@title Set Primary Analysis Segment Info

player_1 = PlayerMetadata(
    session_date='20221002',
    game_pk='662825',
    # demo '444482' for baseball-hitting, '446372' for baseball-pitching
    player_id='446372',
    # set play GUID for the skeleton animation; None defaults to the first play
    play_guid=None, 
    s3_metadata=s3_metadata,
)

analysis_dicts = [get_analysis_dict(player_1)]

In [ ]:
analysis_dicts[0]

In [ ]:
#@title [Optional] Uncomment Below and Set Comparison Analysis Segment Info

# # NOTE: if you uncomment below, make sure to rerun the cell below to download and aggregate from S3

# player_2 = PlayerMetadata(
#     session_date='20221002',
#     game_pk='662825',
#     player_id='592773',
#     # set the play GUID for the skeleton animation; None defaults to the first play
#     play_guid=None, 
#     s3_metadata=s3_metadata,
# )

# analysis_dicts.append(get_analysis_dict(player_2))

# if len(analysis_dicts) > 2:
#     raise ValueError('Currently only 2 analysis dicts are supported - re-run the Primary Segment cell to reset the analysis_dicts')

# print('Now run the cell below to download and aggregate from S3')

In [ ]:
#@title Download and aggregate from S3

for analysis_dict in analysis_dicts:
    load_data_into_analysis_dict(analysis_dict)

In [ ]:
#@title Get Population Inverse Kinematics Data from S3

print(s3_metadata.s3_population_prefix)

print('Downloading population mean...')
pop_mean_df = wr.s3.read_csv([f"{s3_metadata.s3_population_prefix}mean_ik.csv"], index_col=[0])

print('Downloading population standard deviation...')
pop_std_df = wr.s3.read_csv([f"{s3_metadata.s3_population_prefix}std_ik.csv"], index_col=[0])

In [ ]:
#@title Inspect Available Joint Angle Names

joint_angle_names = rtk.get_available_joint_angles(analysis_dicts)
print(f"Available Joint Angles:\nn={len(joint_angle_names)}\n{joint_angle_names}")

In [ ]:
#@title Show Animation

time_column_to_plot = 'time_from_max_hand'  # seconds from max dom hand velo

# joint_angles_to_plot = ['right_elbow', 'left_knee']  # list of joint angles to plot below, from available angles above
joint_angles_to_plot = ['right_elbow']  # list of joint angles to plot below, from available angles above

# set to True to plot the mean joint angle trace across the selection,
# set to False to plot the joint angle trace for the play specified by the play_guid in the analysis_dict from that cell above
plot_joint_angle_mean = False

# set to True to write an html file for each joint angle skeleton animation
write_individual_html = False

In [ ]:
frame_step = 25  # step size between animation frames 
figs = []

for joint_angle_to_plot in joint_angles_to_plot:
    fig = rtk.get_animation(boto3_session, analysis_dicts, pop_mean_df, pop_std_df, time_column_to_plot, joint_angle_to_plot, plot_joint_angle_mean, frame_step=frame_step)
  
    if write_individual_html:
        fig.write_html(f'{joint_angle_to_plot}_animation.html', full_html=True, include_plotlyjs='cdn')

    figs.append(fig)

    fig.show()

In [ ]:
#@title Create Joint Angle HTML Plots

joint_angle_names_no_zeros = [
    angle for angle in joint_angle_names 
    if not np.allclose(pop_mean_df[angle].values, np.zeros(len(pop_mean_df)))
    ]

joint_angle_lists = rtk.list_chunks(joint_angle_names_no_zeros, 3)

plot_colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
               'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
               'rgb(148, 103, 189)']

time_label = 'time_from_max_hand'
figs_angles = []

for joint_angle_list in joint_angle_lists:

    fig = rtk.get_joint_plot(boto3_session, analysis_dicts, pop_mean_df, pop_std_df, time_label, joint_angle_list)
    fig.show()
    figs_angles.append(fig)

In [ ]:
#@title Write Report HTML to Local Folder
rtk.save_figs_to_html(figs+figs_angles)